Loading Data:

In [410]:
import pandas as pd

# Load the Excel file into a DataFrame
data = pd.read_excel("Survey_ICS487_Project.xlsx")


Data Inspection:

In [411]:
# Check the first few rows
print(data.head())

# Get an overview of the data
print(data.info())

# Check for missing values
print(data.isnull().sum())


     Name  Salary (SAR)  Age    State     Sex  Monthly Debt (SAR)  \
0   Ahmed          1332   52   Single  Female              226.42   
1    Naif          2240   51  Married    Male              113.76   
2   salem          2467   22  Married    Male              376.84   
3    Hind          2067   32  Married  Female              187.67   
4  Sultan          1917   60   Single  Female              302.62   

   Elementary Expenses (SAR)        Goal  Number of Children  \
0                    2971.24     Savings                   0   
1                    3369.83     Savings                   2   
2                    7357.43    Spending                   1   
3                    3762.59  Investment                   5   
4                    3424.54    Spending                   0   

  Employment_Status Nationality Budgeting_Rule Financial_Comfortability  \
0        Unemployed       Saudi       50/30/20                  ['yes']   
1        Unemployed       Saudi       50/30/20    

Clean and Preprocess:

In [412]:
# Standardize column names
data.columns = [col.strip().lower().replace(" ", "_") for col in data.columns]
# Check for duplicates and remove them
data = data.drop_duplicates()

# Handle missing values (drop rows or fill them as needed)
data = data.dropna()  # Drop rows with missing values
# Alternatively, fill missing values (example: fill salary with median)
data['salary_(sar)'] = data['salary_(sar)'].fillna(data['salary_(sar)'].median())

# Ensure numeric columns are properly formatted
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Validate ranges for numeric columns
data = data[(data['age'] >= 18) & (data['age'] <= 100)]  # Age should be between 0 and 100
data = data[data['salary_(sar)'] >= 0]  # Salary should not be negative

# Filter out rows where the conditions are met
filtered_data = data[
    (data['nationality'].str.lower() == 'saudi') ]  # Keep only rows where nationality is 'saudi'
filtered_data = filtered_data[
    (filtered_data['financial_comfortability'].str.lower() != 'no') ]  # Exclude rows where financial_comfortability is 'no'
filtered_data = filtered_data[
    (data['goal_progress'].str.lower() != 'no')  # Exclude rows where goal_progress is 'no'
]

# Drop the irrelevant columns
filtered_data = filtered_data.drop(columns=['nationality', 'financial_comfortability', 'goal_progress'])

# Store the filtered data back if needed
data = filtered_data
print(data.tail())



          name  salary_(sar)  age    state     sex  monthly_debt_(sar)  \
1170     bader          2007   28   Single    Male              236.13   
1172    Mugbil          2487   23   Single  Female              738.15   
1173  Khadijah          2994   44  Married    Male              572.44   
1174    Samira          2342   22  Married  Female              179.76   
1175    Salman          1215   27  Married    Male              203.54   

      elementary_expenses_(sar)        goal  number_of_children  \
1170                    3234.27    Spending                   0   
1172                    4716.37    Spending                   0   
1173                   10603.21    Spending                   1   
1174                    8050.24    Spending                   0   
1175                    5343.83  Investment                   1   

     employment_status budgeting_rule  
1170           Student       50/30/20  
1172           Student       50/30/20  
1173        Unemployed       50/

/var/folders/yc/7vmb_7z55z3d92njbdlxxr3m0000gp/T/ipykernel_61078/1195411881.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[


Removing outliers:

In [413]:
import numpy as np

def remove_outliers_iqr(df, column):
    """
    Removes outliers from a column using the IQR method.
    """
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to numeric columns with potential outliers
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data = remove_outliers_iqr(data, col)


# Define age bins and labels
age_bins = [0, 24, 34, 44, 54, float('inf')]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55+']

# Create a new column for age groups
data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, right=True)

data = data.drop(columns=['age'])


Encoding Categorical Variables:

In [414]:
from sklearn.preprocessing import LabelEncoder

# One-hot encode categorical columns
categorical_columns = ['state', 'sex', 'goal', 'employment_status']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Encode the age_group using LabelEncoder
label_encoder = LabelEncoder()
data['age_group_encoded'] = label_encoder.fit_transform(data['age_group'])

# Display the encoding
print("Age Group Encoding:")
for label, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{label}: {encoded}")

data = data.drop(columns=['age_group'])


Age Group Encoding:
18-24: 0
25-34: 1
35-44: 2
45-54: 3
55+: 4


Feature Engineering:

In [415]:
# Create new features
data['debt_to_income_ratio'] = data['monthly_debt_(sar)'] / data['salary_(sar)']
data['savings_ratio'] = data['elementary_expenses_(sar)'] / data['salary_(sar)']

# Handle infinite or NaN values due to division
data['debt_to_income_ratio'] = data['debt_to_income_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)
data['savings_ratio'] = data['savings_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Drop irrelevant columns
data = data.drop(columns=['name'])

data.describe()
data.head()

,salary_(sar),monthly_debt_(sar),elementary_expenses_(sar),number_of_children,budgeting_rule,state_Single,sex_Male,goal_Savings,goal_Spending,employment_status_Student,employment_status_Unemployed,age_group_encoded,debt_to_income_ratio,savings_ratio
0,1332,226.42,2971.24,0,50/30/20,True,False,True,False,False,True,3,0.169985,2.230661
1,2240,113.76,3369.83,2,50/30/20,False,True,True,False,False,True,3,0.050786,1.504388
2,2467,376.84,7357.43,1,50/30/20,False,True,False,True,False,True,0,0.152752,2.982339
5,2318,556.81,5503.34,3,50/30/20,False,True,False,False,False,True,3,0.240211,2.374176
6,2572,629.61,7032.60,5,50/30/20,False,False,False,False,False,True,0,0.244794,2.734292


Saving Data:

In [416]:
# Save cleaned data to a new Excel file
data.to_excel("Cleaned_Survey_ICS487_Project.xlsx", index=False)

# Or save it as a CSV file
data.to_csv("Cleaned_Survey_ICS487_Project.csv", index=False)